In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'\\CHCFPP01\Guest\MRajabi\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime

In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15

Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB  = Project_DIR + '\\2.Project_GDB.gdb'
ContractsDir = Project_DIR + '\\13.Const_AnalysisFiles'
DocumentsDir = Project_DIR + '\\5.Documents'
PythonDir    = Project_DIR + '\\6.Python_Codes'
ReceivedDir  = Project_DIR + '\\7.ReceivedData'
XML_Dir      = ReceivedDir + '\\HSIP Candidate Applications'


# IRIS Dataset
IL_GDB = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\9.0_IL_SafetyDB\IL_SafetyDB.gdb'
IRIS_Route = {y:os.path.join(IL_GDB,'Route_{}'.format(y)) for y in Years}
IRIS_Tab   = {y:os.path.join(IL_GDB,'Att_{}'.format(y)) for y in Years}

# Intersections Dataset
IntPG = IL_GDB + '\\Intersection2014_361999_CleanedUpFall2018'
Intersections = {year:IntPG for year in Years}

#Crash Dataset
CrashData = {y:os.path.join(IL_GDB,'Crash_{}'.format(y)) for y in Years}

Fields = ['AADT',
          'O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH',
          'I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH',
          'LANES','LN_WTH','MED_TYP','MED_WTH','SURF_WTH','URBAN','PG']
RouteID = 'RID'
BMP = 'BMP'
EMP = 'EMP'

# Geocoded projects
HSIP_Int  = Project_GDB + '\\Const_Int'
HSIP_Seg  = Project_GDB + '\\Const_Seg'

SPF_Data     = ReceivedDir  + '\\SPF Coefficients.xlsx'

2018-12-14 09:30:02


In [5]:
for y in range(2005,2017):
    try:
        arcpy.AlterField_management(in_table=IRIS_Route[y],field='INVENTORY',new_field_name='RID',new_field_alias='Route ID')
    except:
        print('Failed: {} {}'.format('RID',y))
    try:
        arcpy.AlterField_management(in_table=IRIS_Route[y],field='BEG_STA',new_field_name='BMP',new_field_alias='Begin Milepost')
    except:
        print('Failed: {} {}'.format('BEG_STA',y))
    try:
        arcpy.AlterField_management(in_table=IRIS_Route[y],field='END_STA',new_field_name='EMP',new_field_alias='End Milepost')
    except:
        print('Failed: {} {}'.format('END_STA',y))    

Failed: RID 2005
Failed: BEG_STA 2005
Failed: END_STA 2005
Failed: RID 2006
Failed: BEG_STA 2006
Failed: END_STA 2006
Failed: RID 2007
Failed: BEG_STA 2007
Failed: END_STA 2007
Failed: RID 2008
Failed: BEG_STA 2008
Failed: END_STA 2008
Failed: RID 2009
Failed: BEG_STA 2009
Failed: END_STA 2009
Failed: RID 2010
Failed: BEG_STA 2010
Failed: END_STA 2010
Failed: RID 2011
Failed: BEG_STA 2011
Failed: END_STA 2011
Failed: RID 2012
Failed: BEG_STA 2012
Failed: END_STA 2012
Failed: RID 2013
Failed: BEG_STA 2013
Failed: END_STA 2013
Failed: RID 2014
Failed: BEG_STA 2014
Failed: END_STA 2014
Failed: RID 2015
Failed: BEG_STA 2015
Failed: END_STA 2015
Failed: RID 2016
Failed: BEG_STA 2016
Failed: END_STA 2016


In [11]:
for y in range(2005,2017):
    if not 'RID' in [f.name for f in arcpy.ListFields(IRIS_Route[y])]:
        print('Add Field', y)
        arcpy.AddField_management(in_table=IRIS_Route[y],field_name='RID',field_alias='Route ID',field_type='TEXT')
    if 'INVENTORY' in [f.name for f in arcpy.ListFields(IRIS_Route[y])]:
        print('Cal Field', y)
        arcpy.CalculateField_management(in_table=IRIS_Route[y],field='RID',expression='!INVENTORY!',expression_type='PYTHON')


Add Field 2006
Cal Field 2006
Add Field 2007
Cal Field 2007
Add Field 2008
Cal Field 2008
Add Field 2009
Cal Field 2009
Add Field 2010
Cal Field 2010
Add Field 2011
Cal Field 2011
Add Field 2012
Cal Field 2012
Add Field 2013
Cal Field 2013
Add Field 2014
Cal Field 2014
Add Field 2015
Cal Field 2015
Add Field 2016
Cal Field 2016


In [5]:
# Current Status of the Projects
def GetCurrentStaus():
    SDF = pd.DataFrame()
    GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
    for GDB in GDBList:
        p = os.path.basename(GDB).split('_')[1]
        status = ''
        try:
            status = json.load(open(os.path.join(ContractsDir,'HSIP_' + p + '_Status.json')))
        except:
            pass
        if len(status)>0:
            df = pd.DataFrame(columns=['ContNum','Type','BaseDataMissing',2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016])
            df['ContNum'] = [p]
            if status['Seg_Input']>0 and status['Int_Input']==0:
                df['Type'] = ['Seg']
            elif status['Seg_Input']==0 and status['Int_Input']>0:
                df['Type'] = ['Int']
            elif status['Seg_Input']>0 and status['Int_Input']>0:
                df['Type'] = ['Corridor']
            elif status['Seg_Input']==0 and status['Int_Input']==0:
                df['Type'] = ['NoLoc']
            by = []
            for y in range(2005,2017):
                if status['BaseInt_{}'.format(y)]==0 or status['BaseRoute_{}'.format(y)]==0 or status['BaseTable_{}'.format(y)]==0:
                    by.append(y)
            df['BaseDataMissing'] = [';'.join([str(y) for y in by])]
            if status['Seg_Input']>0:
                for y in range(2005,2017):
                    if status['SegCrash_{}'.format(y)]:
                        df[y] = ['Crash']
                    else:
                        if status['SegAtt_{}'.format(y)]:
                            df[y] = ['Att']
            if status['Int_Input']>0:
                for y in range(2005,2017):
                    if status['IntCrash_{}'.format(y)]:
                        df[y] = ['Crash']
                    else:
                        if status['IntPoints_{}'.format(y)] and status['IntTables_{}'.format(y)]:
                            df[y] = ['Att']
            #print(p,len(status),df.shape)
            SDF = pd.concat([SDF,df])
    return(SDF)

In [8]:
SDF = GetCurrentStaus()
#for y in range(2005,2017):
#    print(SDF[y].value_counts())
#SDF[SDF[2005]=='Att']
SDF

,ContNum,Type,BaseDataMissing,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,60A25,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60A95,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60C09,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60C14,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60C15,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60D19,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60D20,Int,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60D32,Seg,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60D33,Seg,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60D34,Seg,2005;2006;2007;2008;2009;2010;2011;2012;2013;2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
idf = hsmpy3.common.FCtoDF(HSIP_Int,readGeometry=True,selectedFields=['ContNum'])
sdf = hsmpy3.common.FCtoDF(HSIP_Seg,readGeometry=True,selectedFields=['ContNum'])
df = pd.concat([sdf,idf])
PN = list(set(df.ContNum))
PN.sort()
df = pd.DataFrame(index=PN,columns=['NumInt','NumSeg','Mileage'])
df.index.name = 'ContNum'
df = df.fillna(0)
for i,r in idf.iterrows():
    df.loc[r.ContNum,'NumInt'] += 1
for i,r in sdf.iterrows():
    df.loc[r.ContNum,'NumSeg'] += 1
    df.loc[r.ContNum,'Mileage'] += r.Shape.length/5280.0
print('Total Contracts: {}'.format(df.shape[0]))
print('Total Int: {}, Total Seg: {}, Total Mileage {}'.format(df.NumInt.sum(),df.NumSeg.sum(),df.Mileage.sum()))
print('Int Contracts: {}, Segment Contracts: {}, Int&Seg Contracts: {}'.format(df[(df.NumInt>0) & (df.NumSeg==0)].shape[0],
                                                                               df[(df.NumInt==0) & (df.NumSeg>0)].shape[0],
                                                                               df[(df.NumInt>0) & (df.NumSeg> 0)].shape[0]))

Total Contracts: 335
Total Int: 342, Total Seg: 308, Total Mileage 1606.13169597982
Int Contracts: 143, Segment Contracts: 189, Int&Seg Contracts: 3


In [21]:
# Create GDB Files for each project
print(strftime("%Y-%m-%d %H:%M:%S"))
SP_CreateMDB = []
for p in PN:
    p = str(p)
    SP_CreateMDB.append({'Process':hsmpy3.il.CON_CreateGDBforHSIP(ContractsDir,HSMPY_PATH,p,HSIP_Seg,HSIP_Int),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_CreateMDB = hsmpy3.common.WaitIfNecessary(SP_CreateMDB,10,Frequency = 2)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-13 15:16:57
60A25: 0
60C14: 0
60A95: 0
60C09: 0
60D19: 0
60C15: 0
60D33: 0
60D32: 0
60D20: 0
60D34: 0
60D36: 0
60D37: 0
60D38: 0
60D39: 0
60D40: 0
60D43: 0
60D51: 0
60D52: 0
60D54: 0
60D98: 0
60E64: 0
60E65: 0
60F54: 0
60F62: 0
60F73: 0
60F74: 0
60F76: 0
60F77: 0
60F81: 0
60F82: 0
60F84: 0
60F83: 0
60F87: 0
60G39: 0
60G40: 0
60H72: 0
60I27: 0
60I36: 0
60I37: 0
60I56: 0
60I71: 0
60J90: 0
60K09: 0
60K21: 0
60K23: 0
60K65: 0
60K24: 0
60K67: 0
60K68: 0
60L21: 0
60L22: 0
60L25: 0
60L28: 0
60L43: 0
60L80: 0
60M47: 0
60M48: 0
60M50: 0
60M55: 0
60N18: 0
60P13: 0
60P15: 0
60P20: 0
60P23: 0
60P24: 0
60P63: 0
60P90: 0
60R59: 0
60T14: 0
60T50: 0
60T79: 0
60T80: 0
60T82: 0
60T88: 0
60T90: 0
60T92: 0
60T94: 0
60T95: 0
60T98: 0
60V59: 0
60W05: 0
60V95: 0
60W19: 0
60W91: 0
60W98: 0
60X02: 0
60X32: 0
60X48: 0
62296: 0
62843: 0
62911: 0
62915: 0
62949: 0
63048: 0
63099: 0
63416: 0
63453: 0
63669: 0
63684: 0
63821: 0
63829: 0
64B81: 0
64C30: 0
64C68: 0
64D09: 0
64D51: 0
64D52: 0
64D53: 0
64D76: 0

In [7]:
# Update Status for GDBs
print(strftime("%Y-%m-%d %H:%M:%S"))
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
Years = range(2005,2017)
SP_UpdateStatus =[]
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1]
    #if os.path.exists(AnalysisDir + '\\HSIP_' + p + '_Status.json'):
    #    continue
    SP_UpdateStatus.append({'Process':hsmpy3.il.CON_UpdateStatus(ContractsDir,HSMPY_PATH,GDB,Years,os.path.basename(GDB).split('_')[1]),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_UpdateStatus = hsmpy3.common.WaitIfNecessary(SP_UpdateStatus,10,Frequency = 2)
[p['Process'].wait() for p in SP_UpdateStatus]
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-14 07:22:03
60A95: 0
60D19: 0
60D34: 0
60C15: 0
60C09: 0
60D32: 0
60D33: 0
60A25: 0
60C14: 0
60D20: 0
60D37: 0
60D36: 0
60D38: 0
60D39: 0
60D40: 0
60D43: 0
60D51: 0
60D52: 0
60D54: 0
60D98: 0
60E64: 0
60E65: 0
60F54: 0
60F62: 0
60F73: 0
60F74: 0
60F76: 0
60F77: 0
60F81: 0
60F82: 0
60F83: 0
60F84: 0
60F87: 0
60G39: 0
60G40: 0
60H72: 0
60I27: 0
60I36: 0
60I37: 0
60I56: 0
60I71: 0
60J90: 0
60K09: 0
60K21: 0
60K23: 0
60K24: 0
60K65: 0
60K67: 0
60K68: 0
60L21: 0
60L22: 0
60L25: 0
60L28: 0
60L43: 0
60L80: 0
60M47: 0
60M48: 0
60M50: 0
60M55: 0
60N18: 0
60P13: 0
60P15: 0
60P20: 0
60P23: 0
60P24: 0
60P63: 0
60P90: 0
60R59: 0
60T14: 0
60T50: 0
60T79: 0
60T80: 0
60T82: 0
60T88: 0
60T90: 0
60T92: 0
60T94: 0
60T95: 0
60T98: 0
60V59: 0
60V95: 0
60W05: 0
60W19: 0
60W91: 0
60W98: 0
60X02: 0
60X32: 0
60X48: 0
62296: 0
62843: 0
62911: 0
62915: 0
62949: 0
63048: 0
63099: 0
63416: 0
63453: 0
63669: 0
63684: 0
63821: 0
63829: 0
64B81: 0
64C30: 0
64C68: 0
64D09: 0
64D51: 0
64D52: 0
64D53: 0
64D76: 0

In [13]:
# Add Base Data to each GDB
print(strftime("%Y-%m-%d %H:%M:%S"))
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
Years = range(2005,2017)
SelectionExtent = "0.5 Miles"
SP_AddBaseData =[]
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1]
    status = json.load(open(os.path.join(ContractsDir,'HSIP_' + p + '_Status.json')))
    p_year = []
    for year in Years:
        try:
            if (status['Seg_Input']+status['Int_Input']>0) and (status['BaseRoute_' + str(year)] + status['BaseInt_' + str(year)] + status['BaseTable_' + str(year)]==0):
                p_year.append(year)
        except:
            p_year.append(year)
    if len(p_year)>0:
        SegFC = GDB + '\\Seg_' + p
        IntFC = GDB + '\\Int_' + p
        SP_AddBaseData.append({'Process':hsmpy3.il.CON_AddBaseData(ContractsDir,HSMPY_PATH,[SegFC,IntFC],
                                          p_year,IRIS_Route,IRIS_Tab,Intersections,
                                          GDB,SelectionExtent,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
        SP_AddBaseData = hsmpy3.common.WaitIfNecessary(SP_AddBaseData,3,Frequency = 2)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-12-14 11:33:21
60A25: 0
60A95: 0
60C09: 0
60C14: 0
60C15: 0
60D19: 0
60D20: 0
60D32: 0
60D33: 0
60D36: 0
60D37: 0
60D34: 0
60D38: 0
60D39: 0
60D40: 0
60D43: 0
60D51: 0
60D52: 0
60D54: 0
60D98: 0
60E64: 0
60E65: 0
60F54: 0
60F62: 0
60F73: 0
60F74: 0
60F76: 0
60F77: 0
60F81: 0
60F82: 0
60F83: 0
60F84: 0
60F87: 0
60G39: 0
60G40: 0
60H72: 0
60I27: 0
60I37: 0
60I36: 0
60I56: 0
60I71: 0
60J90: 0
60K09: 0
60K21: 0
60K23: 0
60K24: 0
60K65: 0
60K67: 0
60K68: 0
60L21: 0
60L22: 0
60L25: 0
60L28: 0
60L43: 0
60L80: 0
60M47: 0
60M48: 0
60M50: 0
60M55: 0
60N18: 0
60P13: 0
60P15: 0
60P20: 0
60P23: 0
60P24: 0
60P63: 0
60P90: 0
60R59: 0
60T14: 0
60T50: 0
60T79: 0
60T80: 0
60T82: 0
60T88: 0
60T90: 0
60T92: 0
60T94: 0
60T95: 0
60T98: 0
60V59: 0
60V95: 0
60W05: 0
60W19: 0
60W91: 0
60W98: 0
60X02: 0
60X32: 0
62296: 0
60X48: 0
62843: 0
62911: 0
62915: 0
62949: 0
63048: 0
63099: 0
63416: 0
63453: 0
63669: 0
63684: 0
63821: 0
63829: 0
64B81: 0
64C30: 0
64C68: 0
64D09: 0
64D51: 0
64D52: 0
64D53: 0
64D76: 0

In [ ]:
# Add SiteID to intersections
SDF = GetCurrentStaus()
for p in list(SDF[SDF.Type=='Int']['ContNum']):
    IntFC = ContractsDir + '\\HSIP_{}_GIS.gdb\\Int_{}'.format(p,p)
    if 'SiteID' in arcpy.ListFields(IntFC):
        continue
    arcpy.AddField_management(IntFC,'SiteID','LONG')
    uc = arcpy.UpdateCursor(IntFC)
    i = 0
    for row in uc:
        i += 1
        row.setValue('SiteID',i)
        uc.updateRow(row)
    del uc
    del row
    print(p)

In [ ]:
# Remove intersection buffers from roadways
print(strftime("%Y-%m-%d %H:%M:%S"))
SP_Exc =[]
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1]
    SegFC = GDB + '\\Seg_' + p
    NumSeg = 0
    try: NumSeg = int(str(arcpy.GetCount_management(SegFC)))
    except: pass
    if NumSeg > 0:
        HSIP_Seg_NotExc = SegFC + '_NotExc'
        NumNESeg = 0 
        try: NumNESeg = int(str(arcpy.GetCount_management(HSIP_Seg_NotExc)))
        except: pass
        if NumNESeg==0:
            HSIP_Seg_NotExc = hsmpy3.common.CreateOutPath(MainFile = SegFC,appendix='NotExc',Extension='')
            arcpy.management.CopyFeatures(in_features=SegFC,out_feature_class=HSIP_Seg_NotExc)
            arcpy.management.Delete(SegFC)
        else:
            #print('Delete ' + SegFC)
            arcpy.management.Delete(SegFC)
        
        NumBaseInt = 0
        IntBaseData = GDB + '\\HWY2016_inter'
        try: NumBaseInt = int(str(arcpy.GetCount_management(IntBaseData)))
        except: pass
        if NumBaseInt>0:
            arcpy.env.outputMFlag = "Disabled"
            arcpy.env.outputZFlag = "Disabled"
            arcpy.RecalculateFeatureClassExtent_management (IntBaseData)
            arcpy.RecalculateFeatureClassExtent_management (HSIP_Seg_NotExc)
            SP_Exc.append({'Process':hsmpy3.network.CON_ExtractIntFromSeg(ContractsDir,HSMPY_PATH,HSIP_Seg_NotExc,IntBaseData,'250 Feet',SegFC,Title=p),
                            'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
            SP_Exc = hsmpy3.common.WaitIfNecessary(SP_Exc,MaxOpenProcesses,Frequency = 2)

In [7]:
# Import Roadway and Int Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
XY_Tolerance = "70 Feet"
SP_Att =[]
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
for GDB in GDBList[385:]:
    p = os.path.basename(GDB).split('_')[1]
    #status = json.load(open(os.path.join(ContractsDir,'HSIP_' + p + '_Status.json')))
    #p_year = []
    #for year in Years:
    #    try:
    #        if (status['Seg_Input']+status['Int_Input']>0) and (status['BaseRoute_' + str(year)] + status['BaseInt_' + str(year)] + status['BaseTable_' + str(year)]>0):
    #            if (status['Seg_Input']>0 and not status['SegAtt_'+str(year)]) or (status['Int_Input']>0 and not status['IntPoints_'+str(year)]):
    #                p_year.append(year)
    #    except:
    #        p_year.append(year)
    p_year = range(2005,2017)
    if len(p_year)>0:
        print(p,p_year)
        SegFC = GDB + '\\Seg_' + p
        IntFC = GDB + '\\Int_' + p
        RteFN = {year:os.path.join(GDB,'HWY' + str(year) + '_route') for year in p_year}
        IntFN = {year:os.path.join(GDB,'HWY' + str(year) + '_inter') for year in p_year}
        TabFN = {year:os.path.join(GDB,'HWY' + str(year) + '_table') for year in p_year}
        NumSeg = 0
        NumInt = 0
        try: NumSeg = int(str(arcpy.GetCount_management(SegFC)))
        except: pass
        try: NumInt = int(str(arcpy.GetCount_management(IntFC)))
        except: pass
        if NumSeg > 0:
            arcpy.AddField_management(SegFC,'INVENTORY_I','Text')
            arcpy.CalculateField_management(SegFC,'INVENTORY_I','!INVENTORY!')
            #arcpy.DeleteField_management(SegFC,'INVENTORY')
            RoadwayData = {year:hsmpy3.common.CreateOutPath(MainFile=SegFC,appendix=str(year),Extension='') for year in p_year}
            SP_Att.append({'Process':hsmpy3.network.CON_ImportRoadwayData_Temporal(ContractsDir,HSMPY_PATH,
                                                                                  SegFC,RteFN,TabFN,
                                                                                  Fields,RoadwayData,
                                                                                  RouteID,BMP,EMP,XY_Tolerance,p,"INVENTORY_I"),
                            'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
        if NumInt > 0:
            IntPoints = {year:hsmpy3.common.CreateOutPath(MainFile=IntFC,appendix=str(year) + '_points',Extension='') for year in p_year}
            IntTables = {year:hsmpy3.common.CreateOutPath(MainFile=IntFC,appendix=str(year) + '_tables',Extension='') for year in p_year}
            SP_Att.append({'Process':hsmpy3.network.CON_ImportIntData_Temporal(ContractsDir,HSMPY_PATH,
                                                                       IntFC,IntFN,
                                                                       RteFN,RouteID,BMP,EMP,TabFN,Fields,
                                                                       IntPoints,IntTables,p),
                            'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
        SP_Att = hsmpy3.common.WaitIfNecessary(SP_Att,MaxOpenProcesses,Frequency = 2)    
[p['Process'].wait() for p in SP_Att]
print(strftime("%Y-%m-%d %H:%M:%S"))
SP_UpdateStatus =[]
for sp in SP_Att:
    p = sp['HSIP_ID']
    GDB = os.path.join(ContractsDir,'HSIP_' + p + '.gdb')
    SP_UpdateStatus.append({'Process':hsmpy3.il.CON_UpdateStatus(ContractsDir,HSMPY_PATH,GDB,Years,os.path.basename(GDB).split('_')[1]),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_UpdateStatus = hsmpy3.common.WaitIfNecessary(SP_UpdateStatus,MaxOpenProcesses,Frequency = 2)
[p['Process'].wait() for p in SP_UpdateStatus]
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-29 12:34:40
72E04 range(2005, 2017)
72E31 range(2005, 2017)
72F06 range(2005, 2017)
72F13 range(2005, 2017)
72F24 range(2005, 2017)
72F29 range(2005, 2017)
72F31 range(2005, 2017)
72F50 range(2005, 2017)
72F90 range(2005, 2017)
72F95 range(2005, 2017)
72G04 range(2005, 2017)
72G73 range(2005, 2017)
72G91 range(2005, 2017)
72G93 range(2005, 2017)
72H04 range(2005, 2017)
72E04: 0
72H33 range(2005, 2017)
72E31: 0
72H52 range(2005, 2017)
72F29: 0
72I07 range(2005, 2017)
72F90: 0
72J03 range(2005, 2017)
72F95: 0
74161 range(2005, 2017)
72F06: 0
74205 range(2005, 2017)
72F13: 0
72G73: 0
72G91: 0
72G93: 0
74206 range(2005, 2017)
72F24: 0
72H04: 0
74208 range(2005, 2017)
72F31: 0
74209 range(2005, 2017)
72F50: 0
74300 range(2005, 2017)
72G04: 0
72H33: 0
72H52: 0
74422 range(2005, 2017)
74423 range(2005, 2017)
74425 range(2005, 2017)
74426 range(2005, 2017)
74427 range(2005, 2017)
72I07: 0
74444 range(2005, 2017)
74445 range(2005, 2017)
74446 range(2005, 2017)
72J03: 0
74447 range(2005,

AL11394: 0
AL14006 range(2005, 2017)
AL12016: 0
AL14027 range(2005, 2017)
AL12101: 0
AL14186 range(2005, 2017)
AL12166: 0
AL14231 range(2005, 2017)
AL12213: 0
AL14349 range(2005, 2017)
AL13057: 0
AL14366 range(2005, 2017)
AL13123: 0
AL14382 range(2005, 2017)
AL13171: 0
AL15069 range(2005, 2017)
AL13287: 0
AL14382: 1
AL15254 range(2005, 2017)
AL11487: 0
AL13339: 0
AL15427 range(2005, 2017)
AL13347: 0
AL16012 range(2005, 2017)
AL16190 range(2005, 2017)
AL13375: 0
AL16203 range(2005, 2017)
AL17013 range(2005, 2017)
AL13383: 0
AL17156 range(2005, 2017)
AL13447: 0
AL17199 range(2005, 2017)
AL14006: 0
AL18009 range(2005, 2017)
AL14027: 0
AL18018 range(2005, 2017)
AL14186: 0
AL18054 range(2005, 2017)
AL14231: 0
AL18071 range(2005, 2017)
AL14349: 0
AL18136 range(2005, 2017)
AL14366: 0
CH13046 range(2005, 2017)
AL15069: 0
CH18005 range(2005, 2017)
AL15254: 0
LA108 range(2005, 2017)
AL15427: 0
AL17156: 1
LA110 range(2005, 2017)
LA111 range(2005, 2017)
AL16012: 0
LA112 range(2005, 2017)
AL16190: 

In [5]:
# Add Observed Crashes
#[p['Process'].wait() for p in SP_Att]
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
Seg_Buffer_Distance = '50 Feet'
Int_Buffer_Distance = '250 Feet'
SP_ObsCrash =[]
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
for GDB in GDBList[465:]:
    p = os.path.basename(GDB).split('_')[1]
    #try:
    #    status = json.load(open(os.path.join(ContractsDir,'HSIP_' + p + '_Status.json')))
    #    p_year = []
    #    for year in Years:
    #        try:
    #            if (status['Seg_Input']+status['Int_Input']>0) and (status['BaseRoute_' + str(year)] + status['BaseInt_' + str(year)] + status['BaseTable_' + str(year)]>0):
    #                if (status['Seg_Input']>0 and status['SegAtt_'+str(year)]>0 and not status['SegCrash_' + str(year)]) or (status['Int_Input']>0 and status['IntPoints_'+str(year)]>0 and not status['IntCrash_' + str(year)]):
    #                    p_year.append(year)
    #        except:
    #            pass
    #except:
    p_year = Years
    print(p,p_year)
    if len(p_year)>0:
        Sites = []
        for year in p_year:
            SegFC = GDB + '\\Seg_' + p + '_' + str(year)
            IntFC = GDB + '\\Int_' + p + '_' + str(year) + '_points'
            NumSeg = 0
            NumInt = 0
            try: NumSeg = int(str(arcpy.GetCount_management(SegFC)))
            except: pass
            try: NumInt = int(str(arcpy.GetCount_management(IntFC)))
            except: pass
            if NumSeg > 0:
                SegCrashes = hsmpy3.common.CreateOutPath(MainFile= SegFC,appendix=str('Crash'),Extension='')
                Sites.append({'year':year,'Input':SegFC,'Output':SegCrashes,'Buffer':Seg_Buffer_Distance})
            if NumInt > 0:
                IntCrashes = hsmpy3.common.CreateOutPath(MainFile= IntFC,appendix=str('Crash'),Extension='')
                Sites.append({'year':year,'Input':IntFC,'Output':IntCrashes,'Buffer':Int_Buffer_Distance})
        SP_ObsCrash.append({'Process':hsmpy3.crash.CON_AddObservedCrashes_Temporal(ContractsDir,HSMPY_PATH,Sites,CrashData,p),
                            'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
        SP_ObsCrash = hsmpy3.common.WaitIfNecessary(SP_ObsCrash,MaxOpenProcesses,Frequency = 2)   
[p['Process'].wait() for p in SP_ObsCrash]
print(strftime("%Y-%m-%d %H:%M:%S"))
SP_UpdateStatus =[]
for sp in SP_ObsCrash:
    p = sp['HSIP_ID']
    GDB = os.path.join(ContractsDir,'HSIP_' + p + '.gdb')
    SP_UpdateStatus.append({'Process':hsmpy3.il.CON_UpdateStatus(ContractsDir,HSMPY_PATH,GDB,Years,os.path.basename(GDB).split('_')[1]),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_UpdateStatus = hsmpy3.common.WaitIfNecessary(SP_UpdateStatus,MaxOpenProcesses,Frequency = 2)
[p['Process'].wait() for p in SP_UpdateStatus]
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-04 08:38:16
76D42 range(2005, 2017)
76D63 range(2005, 2017)
76D89 range(2005, 2017)
76E30 range(2005, 2017)
76E31 range(2005, 2017)
76E32 range(2005, 2017)
76D42: 0
76E33 range(2005, 2017)
76D63: 0
76D89: 0
76E35 range(2005, 2017)
76E52 range(2005, 2017)
76E31: 0
76E53 range(2005, 2017)
76E78 range(2005, 2017)
76E32: 0
76F11 range(2005, 2017)
76E30: 0
76F16 range(2005, 2017)
76E33: 0
76F18 range(2005, 2017)
76E35: 0
76F76 range(2005, 2017)
76E52: 0
76F11: 0
76F92 range(2005, 2017)
76F96 range(2005, 2017)
76E53: 0
76F97 range(2005, 2017)
76F98 range(2005, 2017)
76E78: 0
76F18: 0
76F99 range(2005, 2017)
76F16: 0
76F76: 0
76G00 range(2005, 2017)
76F92: 0
76G13 range(2005, 2017)
76F96: 0
76G34 range(2005, 2017)
76G67 range(2005, 2017)
76F97: 0
76G34: 0
76G68 range(2005, 2017)
76F98: 0
76G69 range(2005, 2017)
76G00: 0
76G70 range(2005, 2017)
76G13: 0
76H20 range(2005, 2017)
76F99: 0
76H78 range(2005, 2017)
76G67: 0
76G68: 0
76H79 range(2005, 2017)
76J25 range(2005, 2017)
76G69: 0
76

76J25: 1
76H79: 0
76J27: 0
76J29: 1
76J87: 1
78038: 1
78042: 1
78045: 1
78047: 1
78048: 1
78053: 1
78098: 1
76J28: 0
78113: 1
78116: 1
76J88: 0
78277: 0
78317: 0
78333: 1
78352: 1
78362: 1
78395: 1
78396: 1
78407: 1
78449: 1
78457: 1
78508: 1
78509: 1
78320: 0
78519: 1
78476: 0
78552: 0
78597: 1
78592: 0
78640: 1
85412: 1
85413: 1
85533: 1
85545: 1
85615: 1
85629: 1
85636: 1
85662: 1
87479: 1
85395: 0
87514: 0
87518: 1
87517: 0
87573: 1
87579: 1
89470: 1
89480: 1
89561: 1
89562: 1
89572: 1
89591: 1
91392: 1
87519: 0
91393: 1
87546: 0
91445: 0
91481: 1
91467: 0
91511: 1
91531: 1
93423: 1
93443: 1
93463: 1
93468: 1
93477: 1
93479: 1
93493: 1
93558: 1
91489: 0
93609: 0
95506: 1
93650: 0
97288: 1
97319: 1
97462: 1
97514: 1
97519: 1
97542: 1
98820: 1
98968: 1
98991: 1
95831: 0
99283: 1
97323: 0
99293: 0
99326: 1
99306: 0
99363: 1
99366: 1
99369: 1
99413: 1
99443: 1
99463: 1
99495: 1
99590: 1
AL07152: 1
99341: 0
AL08110: 1
AL08109: 0
AL08125: 0
AL08155: 1
AL08128: 0
AL08203: 1
AL08205: 1
AL0

In [ ]:
# Add Predicted Crashes
#[p['Process'].wait() for p in SP_ObsCrash]
print(strftime("%Y-%m-%d %H:%M:%S"))
SP_PrdCrash = []
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1]
    SP_PrdCrash.append({'Process':hsmpy3.il.CON_Seg_PC_HSIP(ContractsDir,HSMPY_PATH,GDB,Years,SPF_Data,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_PrdCrash = hsmpy3.common.WaitIfNecessary(SP_PrdCrash,15,Frequency = 2)  

2018-09-04 13:17:59
13937: 1
13873: 1
13938: 1
13940: 1
14001: 1
43475: 1
43521: 1
60A25: 1
60A95: 1
60C09: 1
60C14: 1
13684: 0
60D20: 1
60D33: 1
60D34: 1
60D36: 1
60D37: 1
60D38: 1
60D39: 1
60D40: 1
60D54: 1
60E31: 1
60E64: 1
60E65: 1
60F54: 1
60F75: 1
60F76: 1
13872: 0
13570: 0
13677: 0
60C15: 0
60D32: 0
60D19: 0
60D43: 0
60D51: 0
60D98: 0
60D52: 0
60F74: 0
60F73: 0
60F62: 0
60F81: 0
60F77: 0
60K09: 0
60F82: 0
60F83: 0
60F84: 0
60F87: 0
60G39: 0
60G40: 0
60H72: 0
60I27: 0
60I37: 0
60I36: 0
60L22: 0
60I56: 0
60J90: 0
60J88: 0
60L28: 0
60L43: 0
60K21: 0
60K23: 0
60K24: 0
60L25: 0
60K65: 0
60K67: 0
60M55: 0
60K68: 0
60K99: 0
60L21: 0
60M47: 0
60L80: 0
60M48: 0
60M50: 0
60N18: 0
60T24: 0
60P13: 0
60P15: 0
60P20: 0
60T50: 0
60P23: 0
60P24: 0
60P64: 0
60T94: 0
60P63: 0
60P90: 0
60R86: 0
60R59: 0
60T97: 0
60V34: 0
60V42: 0
60T04: 0
60T19: 0
60T14: 0
60V99: 0
60T27: 0
60T92: 0
60T80: 0
60W16: 0
60T90: 0
60W19: 0
60T79: 0
60T95: 0
60W55: 0
60T82: 0
60W90: 0
60T88: 0
60X25: 1
60X34: 1
60X08: 0

In [ ]:
# Create Summary Sheet HSIP
SP_ExcelSheet = []
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1]
    if not int(p) in Cont_DF.ContNum:
        continue
    if not(len(HSIP_DF[HSIP_DF['HSIP ID'] ==  int(p)]['BeforePeriod'].item())>0 and len(HSIP_DF[HSIP_DF['HSIP ID'] ==  int(p)]['AfterPeriod'].item())>0):
        print('{}: Before-After Periods are not defined'.format(p))
        continue
    FN = os.path.join(AnalysisDir,'HSIP_'+str(p) + '.xlsx')
    try: os.remove(FN)
    except: pass
    SP_ExcelSheet.append({'Process':hsmpy3.il.CON_ExcelSheetSummary(ContractsDir,HSMPY_PATH,GDB,Years,FN,HSIP_Data,p,Fields),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_ExcelSheet = hsmpy3.common.WaitIfNecessary(SP_ExcelSheet,5,Frequency = 2) 

In [27]:
Cont_DF = hsmpy3.il.Joined_Contract_DF(HSIP_Joined,WPPS_Ob_DF,WPPS_Pr_DF,ReceivedDir,Years)

In [4]:
# Adjust dates
HSIP_AwDate = []
HSIP_CmDate = []

for i,r in Cont_DF.iterrows():
    hsip_ids = str(r.HSIPIDs).split(';')
    tot_l = []
    tot_c = []
    for hsip in hsip_ids:
        if int(hsip) in list(HSIP_DF.index):
            tot_l.append(HSIP_DF['Award Date'][int(hsip)])
            tot_c.append(HSIP_DF['Completion Date'][int(hsip)])
    tot_l = list(set([d for d in tot_l if not pd.isnull(d)]))
    tot_c = list(set([d for d in tot_c if not pd.isnull(d)]))
    if tot_l!=[]:
        HSIP_AwDate.append(max(tot_l))
    else:
        HSIP_AwDate.append(max([None]))
    if tot_c!=[]:
        HSIP_CmDate.append(min(tot_c))
    else:
        HSIP_CmDate.append(max([None]))
Cont_DF['HSIP_AwardDate'] = HSIP_AwDate
Cont_DF['HSIP_CompDate'] = HSIP_CmDate

In [5]:
Cont_DF['F_AwardDate'] = pd.NaT
Cont_DF['F_CompDate' ] = pd.NaT
Cont_DF['F_AwardDateSrc'] = ''
Cont_DF['F_CompDateSrc' ] = ''

for i,r in Cont_DF.iterrows():
    aw = None
    awSrc = ''
    if not pd.isnull(r.AwardDate):
        if r.AwardDate>datetime(2004,1,1):
            aw = r.AwardDate
            awSrc = 'WPPS'
    if aw is None:
        if not pd.isnull(r.HSIP_AwardDate):
            if r.HSIP_AwardDate>datetime(2004,1,1):
                aw = r.HSIP_AwardDate
                awSrc = 'HSIP'
    cm = None
    cmSrc = ''
    if not pd.isnull(r.CompletionDate):
        if r.CompletionDate>datetime(2004,1,1):
            cm = r.CompletionDate
            cmSrc = 'WPPS'
    if cm is None:
        if not pd.isnull(r.HSIP_CompDate):
            if r.HSIP_CompDate>datetime(2004,1,1):
                cm = r.HSIP_CompDate
                cmSrc = 'HSIP'
    if aw==None and cm!=None and cm>datetime(2007,1,1):
        aw = cm - timedelta(days=(365))
        awSrc = 'CompDate'
    if aw!=None and cm==None and aw<datetime(2014,1,1):
        cm = aw + timedelta(days=(365))
        cmSrc = 'AwardDate'
    Cont_DF.set_value(i,'F_AwardDate',aw)
    Cont_DF.set_value(i,'F_CompDate',cm)
    Cont_DF.set_value(i,'F_AwardDateSrc',awSrc)
    Cont_DF.set_value(i,'F_CompDateSrc',cmSrc)
    

In [6]:
def AddBeforeAfterDates_Contract(HSIP_DF,Years):
    BeforePeriod = []
    AfterPeriod  = []
    ConstPeriod  = []
    for i,hsip in HSIP_DF.iterrows():
        if hsip['F_AwardDate'].year>2003 and hsip['F_AwardDate'].year<2017 and hsip['F_CompDate'].year>2003 and hsip['F_CompDate'].year<2017:
            BeforePeriod.append(';'.join([str(y) for y in Years if y<hsip['F_AwardDate'].year]))
            AfterPeriod.append (';'.join([str(y) for y in Years if y>hsip['F_CompDate'].year]))
            ConstPeriod.append (';'.join([str(y) for y in Years if y>=hsip['F_AwardDate'].year and y<=hsip['F_CompDate'].year]))
        else:
            BeforePeriod.append('')
            AfterPeriod.append('')
            ConstPeriod.append('')
    HSIP_DF['BeforePeriod'] = BeforePeriod
    HSIP_DF['AfterPeriod'] = AfterPeriod
    HSIP_DF['ConstPeriod'] = ConstPeriod
    return(HSIP_DF)
Cont_DF = AddBeforeAfterDates_Contract(Cont_DF,range(2005,2017))

In [7]:
print(Cont_DF.F_AwardDateSrc.value_counts(True))
print(Cont_DF.F_CompDateSrc.value_counts(True))

WPPS        0.616575
HSIP        0.208840
            0.161326
CompDate    0.013260
Name: F_AwardDateSrc, dtype: float64
WPPS         0.466298
             0.348066
HSIP         0.141436
AwardDate    0.044199
Name: F_CompDateSrc, dtype: float64


In [8]:
Cont_DF['HSIPAmount'] = [sum([r[c] for c in ['D20','D21','D28','D10','D15','D22']]) for i,r in Cont_DF.iterrows()]

In [9]:
SLBC = pd.read_excel(Project_DIR+'\\9.QC_Milhouse\\ServiceLife.xlsx',skiprows=3)
SLBC = SLBC[['COUNTERMEASURES','Service Life','CMF','Crash Type Affected by Countermeasures','Source']]
SLBC.columns = ['CntrMsre','ServL','CMF','CrashType','Source']
SLBC = SLBC[SLBC.ServL>0]
SLBC.index = list(SLBC.CntrMsre)

In [10]:
SL = pd.read_excel(Project_DIR+'\\9.QC_Milhouse\\QC_Contracts_FromKyle.xlsx')
SL['ContNum'] = list(SL.index)
SL = SL[['ContNum','Has BC Summary Sheet in the attachments? (Yes,No)','CMF (From BC Summary Sheet)',
         'Service Life (From BC Summary Sheet or ServiceLife File)',
 'Major Countermeasure (If there was no BC Summary Sheet)']]
SL.columns = ['ContNum','HasBCSheet','CMF','ServL','MajCntr']
SL.loc['60F62','CMF'] = np.mean([float(v) for v in SL.loc['60F62','CMF'].split('-')])
SL.loc[(SL.index == '62949') & (SL.CMF == 'Varies'), 'CMF'] = 0.7
SL.loc['64G97','CMF'] = 0.7
SL.loc['76G70','CMF'] = np.mean([float(v) for v in SL.loc['76G70','CMF'].split(',')])
SL.loc[SL.HasBCSheet=='No','CMF'] = list(SLBC.loc[list(SL.loc[SL.HasBCSheet=='No','MajCntr']),'CMF'])
SL.loc[SL.CMF=='Varies','CMF'] = 0.7
SL.loc[SL.MajCntr=='Cannot determine what improvements were made','ServL'] = 15
SL.loc[pd.isnull(SL.ServL),'ServL'] = 15
SL.loc[SL.CMF=='0.64, 0.38','CMF'] = [np.mean([float(x) for x in v.split(',')]) for v in SL.loc[SL.CMF=='0.64, 0.38','CMF']]
SL.CMF = SL.CMF.fillna(0.7)

In [11]:
SL['CMF_ServL'] = [[c,s] for c,s in zip(SL.CMF,SL.ServL)]
def FindAveServL(L):
    #print(L)
    c = [1-p[0] for p in L]
    s = [p[1] for p in L]
    return(np.average(s, weights=c))
df = pd.DataFrame(SL.groupby('ContNum')['CMF_ServL'].aggregate(FindAveServL))
df.columns = ['ServiceLife']
Cont_DF['ServiceLife'] = list(df.loc[list(Cont_DF.ContNum)]['ServiceLife'])
Cont_DF.ServiceLife = Cont_DF.ServiceLife.fillna(15)
Cont_DF.ServiceLife = [int(round(v)) for v in Cont_DF.ServiceLife]
Cont_DF.ServiceLife = Cont_DF.ServiceLife.astype(int)
Cont_DF.to_excel(DocumentsDir + '\\Cont_DF.xlsx')

In [13]:
# Create Summary Sheet Contract
print(strftime("%Y-%m-%d %H:%M:%S"))
SP_ExcelSheet = []
GDBList = [os.path.join(ContractsDir, f) for f in os.listdir(ContractsDir) if os.path.isdir(os.path.join(ContractsDir, f)) and f.split('.')[1]=='gdb' and 'HSIP_' in f.split('.')[0]]    
for GDB in GDBList:
    p = os.path.basename(GDB).split('_')[1]
    if not p in Cont_DF.ContNum:
        continue
    if not(len(Cont_DF[Cont_DF['ContNum'] ==  str(p)]['BeforePeriod'].item())>0 and len(Cont_DF[Cont_DF['ContNum'] ==  str(p)]['AfterPeriod'].item())>0):
        print('{}: Before-After Periods are not defined'.format(p))
        continue
    FN = os.path.join(ContractsDir,'HSIP_'+str(p) + '.xlsx')
    try: os.remove(FN)
    except: pass
    SP_ExcelSheet.append({'Process':hsmpy3.il.CON_ExcelSheetSummary_Contract(ContractsDir,HSMPY_PATH,GDB,Years,FN,DocumentsDir + '\\Cont_DF.xlsx',p,Fields),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_ExcelSheet = hsmpy3.common.WaitIfNecessary(SP_ExcelSheet,8,Frequency = 2) 
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-05 08:31:10
13570: Before-After Periods are not defined
13677: Before-After Periods are not defined
13684: Before-After Periods are not defined
13872: Before-After Periods are not defined
13873: Before-After Periods are not defined
13937: Before-After Periods are not defined
13938: Before-After Periods are not defined
13940: Before-After Periods are not defined
14001: Before-After Periods are not defined
43521: Before-After Periods are not defined
43475: 0
60C14: 1
60C15: 1
60D19: 1
60D20: 1
60A95: 0
60D32: 0
60D36: 1
60D37: 1
60D33: 0
60D34: 0
60D38: 0
60D39: 0
60D51: 1
60D40: 0
60D43: 0
60E31: 1
60A25: 0
60C09: 0
60F73: 0
60F74: 0
60F75: Before-After Periods are not defined
60F76: 0
60D52: 0
60E64: 0
60D54: 0
60E65: 0
60D98: 0
60F83: 0
60F84: 0
60F54: 0
60H72: 0
60F62: 0
60F77: 0
60I37: 0
60F87: 0
60G39: 0
60F81: 0
60G40: 0
60K09: 0
60F82: 0
60K21: 0
60K23: 0
60I27: 0
60K65: 0
60K99: Before-After Periods are not defined
60K67: 0
60K68: 0
60L22: 0
60L25: 0
60L28: 0
60L43: 0
60

74300: 0
74425: 0
74515: Before-After Periods are not defined
74426: 0
74444: 0
74692: Before-After Periods are not defined
74711: Before-After Periods are not defined
74712: Before-After Periods are not defined
74713: Before-After Periods are not defined
74723: Before-After Periods are not defined
74742: Before-After Periods are not defined
74748: Before-After Periods are not defined
74749: Before-After Periods are not defined
74787: Before-After Periods are not defined
74814: Before-After Periods are not defined
74816: Before-After Periods are not defined
74445: 0
74427: 0
74446: 0
74447: 0
74510: 0
76408: 0
74552: 0
76691: 0
74610: 0
76A18: 0
76994: 0
76959: 0
76A34: 0
76A20: 0
76A28: 0
76A76: 0
76B22: 0
76B17: 0
76B25: 0
76B23: 0
76B34: 0
76B48: 0
76B57: 0
76B79: 0
76C29: 0
76C14: 0
76A27: 0
76C30: 0
76C64: 0
76C31: 0
76C32: 0
76D04: 0
76D39: 0
76D14: 0
76D38: 0
76D36: 0
76D41: 0
76D40: 0
76D63: 0
76D42: 0
76E30: 0
76D89: 0
76F11: Before-After Periods are not defined
76E32: 0
76E33

In [6]:
# Append Results to DF
XLSPath = AnalysisDir
XLSList = [os.path.join(XLSPath, f) for f in os.listdir(XLSPath) if os.path.isfile(os.path.join(XLSPath, f)) and f.split('.')[1]=='xlsx' and 'HSIP_' in f.split('.')[0]]
for xls in XLSList:
    p = int(os.path.basename(xls).split('_')[1].split('.')[0])
    df10 = pd.read_excel(xls,'BCSummary')
    df10.index = df10.Period
    #display(df10)
    for per in ['Before','After']:
        for sev in ['K','A','B']:
            for met in ['OC','EC']:
                try:
                    HSIP_DF.set_value(p,'_'.join([per[0],sev,met]),df10.loc[per]['_'.join([sev,met])].item())
                except:pass
    for c in ['EUAC','EUAB_OC','EUAB_EC','BC_OC','BC_EC']:
        try:
            HSIP_DF.set_value(p,c,df10.loc['After'][c].item())
        except:pass